In [1]:
import queue

import numpy as np

In [2]:
def load_data(input_id=''): 
    X = np.loadtxt("input"+input_id+".txt", dtype='str')
    X = np.array([list(x) for x in X])
    return X


def search_start(X):
    r, c = np.where(X=='S')
    return [r[0], c[0]]


symbols = ['S', '|', '-', 'L', 'J', '7', 'F']
def connected_tiles(X, t):
    r, c = t
    symbol = X[r][c]
    north = [r-1, c]
    east  = [r, c+1]
    south = [r+1, c]
    west  = [r, c-1]
    if symbol == 'S':
        tiles = [north, east, south, west]
    elif symbol == '|':
        tiles = [north, south]
    elif symbol == '-':
        tiles = [west, east]
    elif symbol == 'L':
        tiles = [north, east]
    elif symbol == 'J':
        tiles = [north, west]
    elif symbol == '7':
        tiles = [south, west]
    elif symbol == 'F':
        tiles = [south, east]
    else:
        tiles = []
    return tiles


def is_connected(tile1, tile2):
    ''' if two tiles are connected by pipe '''
    result = False 
    if (tile1 in connected_tiles(X, tile2)) and (tile2 in connected_tiles(X, tile1)):
        result = True
    return result


def tiles_next(X, tile):
    tiles = [tile_ for tile_ in connected_tiles(X, tile) if is_connected(tile, tile_)]
    return tiles


def go_next(X, tile, tile_previous):
    tiles = tiles_next(X, tile)
    tiles.remove(tile_previous)
    if len(tiles) == 1:
        tile_next = tiles[0]
    else:
        print(f"next tile can be one of {tiles}.")
        return -1
    return tile_next, tile


def get_path(X):
    paths = []
    n_step  = 0
    t_start = search_start(X)
    #print(f"start from {t_start}")
    paths.append([n_step, t_start])
    
    ## forward one step.
    tiles = tiles_next(X, t_start)
    #print(f"{t_start} can go to: {tiles}")
    tile_previous = t_start
    tile = tiles[1]
    n_step += 1
    paths.append([n_step, tile])
    #print(f"{t_start}-->{tile}")
    
    ## loop until coming back to the starting point.
    while not tile==t_start:
        tile, tile_previous = go_next(X, tile, tile_previous)
        n_step += 1
        paths.append([n_step, tile])
        #print(f"{tile_previous}-->{tile}")

    return paths


def split_path(paths):
    n_steps = paths[-1][0]
    for path in paths:
        path[0] = np.min([path[0], n_steps - path[0]])
    return paths
    

def disp_path(X, paths):
    X_path = np.copy(X)
    for path in paths:
        step, tile = path
        X_path[tile[0]][tile[1]] = step
    return X_path

## part 1

In [3]:
## load data
X = load_data(input_id='')

## find the loop.
paths = get_path(X)

#disp_path(X, paths)

## furthest tile can be reached by 2 ways.
paths = split_path(paths)

## calculate the number of steps to reach the furthest file.
x = [path[0] for path in paths]
print(f"to get from the starting position to the point farthest from the starting position: {np.max(x)}")

to get from the starting position to the point farthest from the starting position: 6812


## part 2

In [4]:
corners = ['L', 'J', '7', 'F']
edges   = ['|', '-']

def disp_path2(X, paths):
    ''' except for the main loop, replace everything into '.' '''
    X_path = np.copy(X)
    path_tiles = [path[1] for path in paths]
    n_rows, n_cols = np.shape(X)
    for r in range(n_rows):
        for c in range(1, n_cols):
            if not [r, c] in path_tiles:
                X_path[r][c] == '.'
    return X_path


def is_corner(X, tile):
    tiles = tiles_next(X, tile)
    x = np.array([np.array(tile) - np.array(t) for t in tiles])
    result = True
    if np.all(np.sum(x, axis=0)==0):
        result = False
    return result

In [59]:
## load data
X = load_data(input_id='2b_')

if is_corner(X, search_start(X)):
    corners.append('S')
else:
    edges.append('S')

In [60]:
## find the loop.
paths = get_path(X)
paths = split_path(paths)
X_ = disp_path2(X, paths)
X_

array([['.', 'F', '-', '-', '-', '-', '7', 'F', '7', 'F', '7', 'F', '7',
        'F', '-', '7', '.', '.', '.', '.'],
       ['.', '|', 'F', '-', '-', '7', '|', '|', '|', '|', '|', '|', '|',
        '|', 'F', 'J', '.', '.', '.', '.'],
       ['.', '|', '|', '.', 'F', 'J', '|', '|', '|', '|', '|', '|', '|',
        '|', 'L', '7', '.', '.', '.', '.'],
       ['F', 'J', 'L', '7', 'L', '7', 'L', 'J', 'L', 'J', '|', '|', 'L',
        'J', '.', 'L', '-', '7', '.', '.'],
       ['L', '-', '-', 'J', '.', 'L', '7', '.', '.', '.', 'L', 'J', 'S',
        '7', 'F', '-', '7', 'L', '7', '.'],
       ['.', '.', '.', '.', 'F', '-', 'J', '.', '.', 'F', '7', 'F', 'J',
        '|', 'L', '7', 'L', '7', 'L', '7'],
       ['.', '.', '.', '.', 'L', '7', '.', 'F', '7', '|', '|', 'L', '7',
        '|', '.', 'L', '7', 'L', '7', '|'],
       ['.', '.', '.', '.', '.', '|', 'F', 'J', 'L', 'J', '|', 'F', 'J',
        '|', 'F', '7', '|', '.', 'L', 'J'],
       ['.', '.', '.', '.', 'F', 'J', 'L', '-', '7', '.', '|', '

In [64]:
n_rows, n_cols = np.shape(X_)

N_cross = np.zeros(np.shape(X_)).astype('int')
# n_cross = 0
# for r in range(1, n_rows):
#     if (not X_[r][0] in ['.', '|']):
#     #if (X_[r][0] != X_[r-1][0]) and (X_[r][0] != '|') and (X_[r-1][0] != '|'):
#         n_cross += 1
#     N_cross[r][0] = n_cross

for r in range(n_rows):
    s = ''.join(X_[r][:])
    print(f"{s}")
    #n_cross = N_cross[r][0]
    for c in range(0, n_cols): 
        s_ = s[:c+1].replace('.', '').replace('-', '')
        N_cross[r][c] = len(s_) % 2
        #print(f"{s_}")
N_cross

.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...


array([[0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1],
       [0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0],
       [1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0]])

In [65]:
X_io = np.copy(X_)
for r in range(n_rows):
    for c in range(n_cols):
        if (N_cross[r][c] == 1) and (X_[r][c] not in symbols):
            X_io[r][c] = 'I'

def disp_matrix(X):
    n_rows, n_cols = np.shape(X)
    for r in range(n_rows):
        s = ''.join(X[r][:])
        print(f"{s}")

disp_matrix(X_io)

.F----7F7F7F7F-7....
.|F--7||||||||FJIIII
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|IL7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...
